# Categorical 12.1

In [1]:
import pandas as pd
import numpy as np

In [2]:
fruits = ["apple", "orange", "apple", "apple"] * 2

In [3]:
N = len(fruits)

In [4]:
df = pd.DataFrame({'fruit': fruits,
                   'basket_id': np.arange(N),
                   'count': np.random.randint(3, 15, size=N),
                   'weight': np.random.uniform(0, 4, size=N)},
                   columns= ['basket_id', 'fruit', 'count', 'weight'])

In [5]:
df

,basket_id,fruit,count,weight
0,0,apple,10,2.938040
1,1,orange,5,3.336775
2,2,apple,6,1.638712
3,3,apple,8,0.039897
4,4,apple,3,3.256876
5,5,orange,4,3.934356
6,6,apple,10,0.905301
7,7,apple,10,3.211900


In [6]:
fruit_cat = df['fruit'].astype('category')

In [7]:
fruit_cat

0     apple
1    orange
2     apple
3     apple
4     apple
5    orange
6     apple
7     apple
Name: fruit, dtype: category
Categories (2, object): ['apple', 'orange']

In [8]:
c = fruit_cat.values

type(c)

pandas.core.arrays.categorical.Categorical

In [9]:
c.categories

Index(['apple', 'orange'], dtype='object')

In [10]:
c.codes

array([0, 1, 0, 0, 0, 1, 0, 0], dtype=int8)

In [11]:
df['fruit'] = df['fruit'].astype('category')

In [12]:
df.dtypes

basket_id       int32
fruit        category
count           int32
weight        float64
dtype: object

### Trying another way to create Categorical

In [13]:
my_categories = pd.Categorical(['foo', 'bar', 'baz', 'foo', 'bar'])

my_categories

['foo', 'bar', 'baz', 'foo', 'bar']
Categories (3, object): ['bar', 'baz', 'foo']

In [14]:
categories = ['foo', 'bar', 'baz']

codes = [0, 1, 2, 0, 0, 1]

In [15]:
my_cats_2 = pd.Categorical.from_codes(codes, categories)

In [16]:
my_cats_2

['foo', 'bar', 'baz', 'foo', 'foo', 'bar']
Categories (3, object): ['foo', 'bar', 'baz']

In [17]:
ordered_cat = pd.Categorical.from_codes(codes, categories, ordered=True)

In [18]:
ordered_cat

['foo', 'bar', 'baz', 'foo', 'foo', 'bar']
Categories (3, object): ['foo' < 'bar' < 'baz']

In [19]:
c.codes

array([0, 1, 0, 0, 0, 1, 0, 0], dtype=int8)

## Categoricals processing

In [20]:
np.random.seed(12345)

In [21]:
draws = np.random.randn(1000)

In [22]:
draws[:5]

array([-0.20470766,  0.47894334, -0.51943872, -0.5557303 ,  1.96578057])

In [23]:
bins = pd.qcut(draws, 4)

In [24]:
bins

[(-0.684, -0.0101], (-0.0101, 0.63], (-0.684, -0.0101], (-0.684, -0.0101], (0.63, 3.928], ..., (-0.0101, 0.63], (-0.684, -0.0101], (-2.9499999999999997, -0.684], (-0.0101, 0.63], (0.63, 3.928]]
Length: 1000
Categories (4, interval[float64, right]): [(-2.9499999999999997, -0.684] < (-0.684, -0.0101] < (-0.0101, 0.63] < (0.63, 3.928]]

In [25]:
bins = pd.qcut(draws, 4, labels=['Q1', 'Q2', 'Q3', 'Q4'])

In [26]:
bins

['Q2', 'Q3', 'Q2', 'Q2', 'Q4', ..., 'Q3', 'Q2', 'Q1', 'Q3', 'Q4']
Length: 1000
Categories (4, object): ['Q1' < 'Q2' < 'Q3' < 'Q4']

In [27]:
bins.codes[:10]

array([1, 2, 1, 1, 3, 3, 2, 2, 3, 3], dtype=int8)

In [28]:
bins = pd.Series(bins, name='quatile')

In [29]:
results = (pd.Series(draws)
          .groupby(bins, observed=False)
          .agg(['count', 'min', 'max'])
          .reset_index())

In [30]:
results

,quatile,count,min,max
0,Q1,250,-2.949343,-0.685484
1,Q2,250,-0.683066,-0.010115
2,Q3,250,-0.010032,0.628894
3,Q4,250,0.634238,3.927528


In [31]:
results['quatile']

0    Q1
1    Q2
2    Q3
3    Q4
Name: quatile, dtype: category
Categories (4, object): ['Q1' < 'Q2' < 'Q3' < 'Q4']

In [32]:
pd.Series(draws).groupby(bins, observed=False).agg(['count', 'min', 'max']).reset_index()

,quatile,count,min,max
0,Q1,250,-2.949343,-0.685484
1,Q2,250,-0.683066,-0.010115
2,Q3,250,-0.010032,0.628894
3,Q4,250,0.634238,3.927528


## Better performance

In [33]:
N = 10000000

In [34]:
draws = pd.Series(np.random.randn(N))

In [35]:
labels = pd.Series(['foo', 'bar', 'baz', 'qux']) * (N // 4)

In [36]:
categories = labels.astype('category')

In [37]:
labels.memory_usage(deep=True)

30000328

In [38]:
categories.memory_usage(deep=True)

30000504

## methods for categorized data

In [39]:
s = pd.Series(['a', 'b', 'c', 'd'] * 2)

In [40]:
cat_s = s.astype('category')

In [41]:
cat_s

0    a
1    b
2    c
3    d
4    a
5    b
6    c
7    d
dtype: category
Categories (4, object): ['a', 'b', 'c', 'd']

In [42]:
cat_s.cat.codes

0    0
1    1
2    2
3    3
4    0
5    1
6    2
7    3
dtype: int8

In [43]:
actual_categories = ['a', 'b', 'c', 'd', 'e']

In [44]:
cat_s2 = cat_s.cat.set_categories(actual_categories)

In [45]:
cat_s2

0    a
1    b
2    c
3    d
4    a
5    b
6    c
7    d
dtype: category
Categories (5, object): ['a', 'b', 'c', 'd', 'e']

In [46]:
cat_s.value_counts()

a    2
b    2
c    2
d    2
Name: count, dtype: int64

In [47]:
cat_s2.value_counts()

a    2
b    2
c    2
d    2
e    0
Name: count, dtype: int64

In [48]:
cat_s3 = cat_s[cat_s.isin(['a', 'b'])]

In [49]:
cat_s3

0    a
1    b
4    a
5    b
dtype: category
Categories (4, object): ['a', 'b', 'c', 'd']

In [50]:
cat_s3.cat.remove_unused_categories()

0    a
1    b
4    a
5    b
dtype: category
Categories (2, object): ['a', 'b']

# Advanced use of GroupBy

In [51]:
df = pd.DataFrame({'key': ['a', 'b', 'c'] * 4, 
                   'value': np.arange(12.)})


In [52]:
df

,key,value
0,a,0.0
1,b,1.0
2,c,2.0
3,a,3.0
4,b,4.0
5,c,5.0
6,a,6.0
7,b,7.0
8,c,8.0
9,a,9.0


In [53]:
g = df.groupby('key').value

In [54]:
g.mean()

key
a    4.5
b    5.5
c    6.5
Name: value, dtype: float64

In [56]:
g.transform(lambda x: x.mean()) # Geramos uma Series no mesmo formato que df['value']

0     4.5
1     5.5
2     6.5
3     4.5
4     5.5
5     6.5
6     4.5
7     5.5
8     6.5
9     4.5
10    5.5
11    6.5
Name: value, dtype: float64

In [58]:
g.transform('mean') # we can also use with a alias

0     4.5
1     5.5
2     6.5
3     4.5
4     5.5
5     6.5
6     4.5
7     5.5
8     6.5
9     4.5
10    5.5
11    6.5
Name: value, dtype: float64

In [59]:
g.transform(lambda x: x * 2)

0      0.0
1      2.0
2      4.0
3      6.0
4      8.0
5     10.0
6     12.0
7     14.0
8     16.0
9     18.0
10    20.0
11    22.0
Name: value, dtype: float64

In [60]:
g.transform(lambda x: x.rank(ascending=False))

0     4.0
1     4.0
2     4.0
3     3.0
4     3.0
5     3.0
6     2.0
7     2.0
8     2.0
9     1.0
10    1.0
11    1.0
Name: value, dtype: float64

In [61]:
# using a function on the transform
def normalize(x):

    return (x - x.mean()) /x.std()

In [62]:
g.transform(normalize)

0    -1.161895
1    -1.161895
2    -1.161895
3    -0.387298
4    -0.387298
5    -0.387298
6     0.387298
7     0.387298
8     0.387298
9     1.161895
10    1.161895
11    1.161895
Name: value, dtype: float64

In [63]:
g.apply(normalize)

key    
a    0    -1.161895
     3    -0.387298
     6     0.387298
     9     1.161895
b    1    -1.161895
     4    -0.387298
     7     0.387298
     10    1.161895
c    2    -1.161895
     5    -0.387298
     8     0.387298
     11    1.161895
Name: value, dtype: float64

In [64]:
g.transform('mean')

0     4.5
1     5.5
2     6.5
3     4.5
4     5.5
5     6.5
6     4.5
7     5.5
8     6.5
9     4.5
10    5.5
11    6.5
Name: value, dtype: float64

In [65]:
normalized = (df['value'] - g.transform('mean')) / g.transform('std')

In [66]:
normalized

0    -1.161895
1    -1.161895
2    -1.161895
3    -0.387298
4    -0.387298
5    -0.387298
6     0.387298
7     0.387298
8     0.387298
9     1.161895
10    1.161895
11    1.161895
Name: value, dtype: float64